# SQL

Компания купила крупный сервис для чтения книг по подписке. Моя первая задача как аналитика — проанализировать базу данных.
В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

## Цель

Целью исследования является анализ данных о книжном рынке, содержащихся в предлагаемой базе данных, проведенный анализ должен помочь сформулировать ценностное предложение для нового продукта.

В нашем распоряжении информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. 
База данных, содержащая таблицы `books`, `authors`, `publishers`, `ratings` и `reviews`.

## Исследование таблиц

In [1]:
# импортируем библиотеки
import pandas as pd 
from sqlalchemy import create_engine

In [2]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net', 
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'], 
 db_config['pwd'], 
 db_config['host'], 
 db_config['port'], 
 db_config['db']) 
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

In [3]:
def query(sql):
    return pd.io.sql.read_sql(sql, con = engine)

Исследуем таблицы, выведем первые строки - узнаем содержание колонок, узнаем объем данных.

### Таблица `books`

In [4]:
sql='''
SELECT *
FROM books b
LIMIT 3
'''
query(sql)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135


В таблице `books` представлены идентификаторы книг и авторов, названия книг, количество страниц, дату издания, а также идентификатор издательства.

In [5]:
sql='''
SELECT count(b.book_id) AS cnt
FROM books b
'''
query(sql)
print(f'Таблица `books` содержит {pd.io.sql.read_sql(sql, con = engine).iloc[0,0]} строк.')

Таблица `books` содержит 1000 строк.


### Таблица `authors`

In [6]:
sql='''
SELECT *
FROM authors a
LIMIT 3
'''
query(sql)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie


Таблица `authors` содержит идентификаторы авторов и имена.

In [7]:
sql='''
SELECT count(a.author_id) AS cnt
FROM authors a
'''
query(sql)
print(f'Таблица `authors` содержит {pd.io.sql.read_sql(sql, con = engine).iloc[0,0]} строк.')

Таблица `authors` содержит 636 строк.


### Таблица `publishers`

In [8]:
sql='''
SELECT *
FROM publishers p
LIMIT 3
'''
query(sql)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books


Таблица `publishers` содержит информацию об идентификаторах и названиях издательств.

In [9]:
sql='''
SELECT count(p.publisher_id) AS cnt
FROM publishers p
'''
query(sql)
print(f'Таблица `publishers` содержит {pd.io.sql.read_sql(sql, con = engine).iloc[0,0]} строк.')

Таблица `publishers` содержит 340 строк.


### Таблица `ratings`

In [10]:
sql='''
SELECT *
FROM ratings r
LIMIT 3
'''
query(sql)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5


Таблица `ratings` содержит информацию об идентификаторах оценок книг, а также имя пользователя и поставленная этим пользователем оценка.

In [11]:
sql='''
SELECT count(r.rating_id) AS cnt
FROM ratings r
'''
query(sql)
print(f'Таблица `ratings` оценок содержит {pd.io.sql.read_sql(sql, con = engine).iloc[0,0]} строк.')

Таблица `ratings` оценок содержит 6456 строк.


### Таблица `reviews`

In [12]:
sql='''
SELECT *
FROM reviews rev
LIMIT 3
'''
query(sql)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...


Таблица `reviews` содержит информацию об идентификаторах обзоров и книг, а также имя пользователя и текст обзора.

In [13]:
sql='''
SELECT count(rev.review_id) AS cnt
FROM reviews rev
'''
query(sql)
print(f'Таблица `reviews` содержит {pd.io.sql.read_sql(sql, con = engine).iloc[0,0]} строк.')

Таблица `reviews` содержит 2793 строк.


## Задания

### Посчитайте, сколько книг вышло после 1 января 2000 года

In [14]:
sql = '''SELECT COUNT(*) 
FROM books
WHERE publication_date > '2000-01-01';
'''
query(sql)

print(f'После 1 января 2000 года вышло {pd.io.sql.read_sql(sql, con = engine).iloc[0,0]} книг')

После 1 января 2000 года вышло 819 книг


### Для каждой книги посчитайте количество обзоров и среднюю оценку

In [15]:
sql = '''WITH average_rating AS
  (SELECT b.book_id,
          title,
          avg(r.rating) AS average_rating
   FROM books b
   LEFT JOIN ratings r ON r.book_id = b.book_id
   GROUP BY b.book_id),
     count_reviews AS
  (SELECT b.book_id,
          title,
          count(rev.review_id) AS count_reviews
   FROM books b
   LEFT JOIN reviews rev ON rev.book_id = b.book_id
   GROUP BY b.book_id)
SELECT a.book_id,
       a.title, 
       c.count_reviews,
       a.average_rating
FROM average_rating a
JOIN count_reviews c ON a.book_id = c.book_id
ORDER BY a.average_rating DESC
LIMIT 3
'''
query(sql)

,book_id,title,count_reviews,average_rating
0,17,A Dirty Job (Grim Reaper #1),4,5.0
1,20,A Fistful of Charms (The Hollows #4),2,5.0
2,55,A Woman of Substance (Emma Harte Saga #1),2,5.0


Для каждой книги определены количество обзоров и среднюя оценка.
Отсортировав данные можно видеть, что максимальной оценкой является 5, а минимальной - 1.5.

### Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры

In [16]:
sql = '''SELECT p.publisher_id,
       p.publisher,
       count(book_id) AS cnt
FROM books b
JOIN publishers p ON p.publisher_id = b.publisher_id
WHERE num_pages > 50
GROUP BY p.publisher_id,
         p.publisher
HAVING count(book_id) =
  (SELECT max(cnt)
   FROM
     (SELECT count(book_id) AS cnt
      FROM books b
      WHERE num_pages > 50
      GROUP BY b.publisher_id) AS t)
'''
query(sql)

,publisher_id,publisher,cnt
0,212,Penguin Books,42


Наибольшее число книг толще 50 страниц (42 штуки) выпустило издательство Penguin Books.

### Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками

In [17]:
sql = '''SELECT b.author_id, a.author, ROUND(AVG(r.rating_avg), 2) AS rating_avg
  FROM books b RIGHT JOIN (SELECT book_id, AVG(rating) AS rating_avg
          FROM ratings
          GROUP BY book_id
          HAVING COUNT(rating_id) >= 50) r
          ON b.book_id = r.book_id
        LEFT JOIN authors a ON b.author_id = a.author_id
GROUP BY b.author_id, a.author
ORDER BY AVG(r.rating_avg)DESC
LIMIT 1
'''
query(sql)

,author_id,author,rating_avg
0,236,J.K. Rowling/Mary GrandPré,4.28


Автором с самой высокой средней оценкой книг (которые имеют больше 50 оценок) является Joanne Rowling.

### Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок

In [18]:
sql = '''WITH users_more_50 AS
  (SELECT r.username
   FROM ratings r
   GROUP BY r.username
   HAVING (count(r.rating_id) > 50))
SELECT avg(t.cnt) AS avg_review
FROM
  (SELECT count(rev.review_id) AS cnt
   FROM reviews rev
   WHERE rev.username in
       (SELECT *
        FROM users_more_50)
   GROUP BY rev.username) AS t
'''
query(sql)
print(f'Среднее количество обзоров пользователей, поставивших более 50 оценок равно {pd.io.sql.read_sql(sql, con = engine).iloc[0,0]} обзора.')

Среднее количество обзоров пользователей, поставивших более 50 оценок равно 24.333333333333332 обзора.


## Выводы

В ходе исследования базы данных были изучены таблицы, содержащие информацию о книгах: об авторах, издательствах, оценках и обзорах. При анализе данных были определены следующие факты:

- в базе данных хранится информация о 1000 книг, 636 авторах, 340 издательствах, 6456 пользовтельских оценках и о 2793 пользовательских обзорах на книги;
- 819 книг из 1000 были изданы после 1 января 2000 года;
- для каждой книги посчитано количество обзоров и средняя оценка;
- наибольшее количество книг толще 50 страниц выпустило издательство Penguin Books;
- автором с самой высокой оценкой книг (из книг с 50 и более оценками) является Diana Gabaldon;
- пользователи, поставившие более 50 оценок, в среднем делают по 24 обзора.